# Limpieza del Data

In [156]:
import pandas as pd
import string
import re 

In [157]:
#Cargamos los datas
data1= pd.read_csv('phishing_site_urls.csv') # kaggle
data2= pd.read_csv('URL dataset.csv') #data phishing

In [158]:
# Obtenemos informacion rapida de los datas
data1.info()
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549346 entries, 0 to 549345
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     549346 non-null  object
 1   Label   549346 non-null  object
dtypes: object(2)
memory usage: 8.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     450176 non-null  object
 1   type    450176 non-null  object
dtypes: object(2)
memory usage: 6.9+ MB


In [159]:
# Verificar valores nulos en cada columna 
valores_nulos_data1 = data1.isnull().sum() 
valores_nulos_data2 = data2.isnull().sum() 
print("Valores nulos en data1:\n", valores_nulos_data1) 
print("\nValores nulos en data2:\n", valores_nulos_data2) 


Valores nulos en data1:
 URL      0
Label    0
dtype: int64

Valores nulos en data2:
 url     0
type    0
dtype: int64


In [160]:
# Verificar espacios vacíos en cada columna 
espacios_vacios_data1 = data1.applymap(lambda x: isinstance(x, str) and x.strip() == '').sum() 
espacios_vacios_data2 = data2.applymap(lambda x: isinstance(x, str) and x.strip() == '').sum()
print("\nEspacios vacíos en data1:\n", espacios_vacios_data1) 
print("\nEspacios vacíos en data2:\n", espacios_vacios_data2)

C:\Users\Marisol\AppData\Local\Temp\ipykernel_3864\2263928669.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  espacios_vacios_data1 = data1.applymap(lambda x: isinstance(x, str) and x.strip() == '').sum()
C:\Users\Marisol\AppData\Local\Temp\ipykernel_3864\2263928669.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  espacios_vacios_data2 = data2.applymap(lambda x: isinstance(x, str) and x.strip() == '').sum()



Espacios vacíos en data1:
 URL      0
Label    0
dtype: int64

Espacios vacíos en data2:
 url     0
type    0
dtype: int64


In [161]:
# Exploramos los valores unicos de las columnas 'Label' y 'type'
valores = data1['Label'].unique()
print(valores)

valores = data2['type'].unique()
print(valores)

['bad' 'good']
['legitimate' 'phishing']


In [ ]:
# Eliminamos filas duplicadas 
for df in [data1, data2]:
    duplicados= df.duplicated().sum()
    print(f"Hay {duplicados} filas duplicadas")
    df.drop_duplicates(inplace=True)
    eliminado=duplicados
    print(f"Se ha eliminado {eliminado} filas")
    
    print(f"Datas sin filas duplicadas")


Hay 42150 filas duplicadas
Se ha eliminado 42150 filas
Datas sin filas duplicadas
Hay 0 filas duplicadas
Se ha eliminado 0 filas
Datas sin filas duplicadas


# Concatenamos datas

In [163]:
#renombrar columnas 
data1.rename(columns={'URL': 'url', 'Label': 'status'}, inplace=True)
data2.rename(columns={'type': 'status'}, inplace=True)


#unir datas
datas=pd.concat([data1,data2], ignore_index=True)

#actualizamos las etiquetas del data1
datas['status']=datas['status'].replace({'bad': '-1', 'good': '1', 'phishing': '-1', 'legitimate':'1'})
print("Las datas han sido unidas")
print(datas.columns)

Las datas han sido unidas
Index(['url', 'status'], dtype='object')


In [164]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957372 entries, 0 to 957371
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     957372 non-null  object
 1   status  957372 non-null  object
dtypes: object(2)
memory usage: 14.6+ MB


In [165]:
# La columna status tiene como valores de 1=legitimate, -1=phishing
conteo_datas= datas['status'].value_counts()
print(conteo_datas)


status
1     738635
-1    218737
Name: count, dtype: int64


In [166]:

# Funcion para verificar si la url usa ciertos prefijos
def has_prefix(url, prefix): 
    return 1 if url.startswith(prefix) else 0 
# Crear columnas para cada prefijo en el DataFrame 
datas['has_https'] = datas['url'].apply(lambda x: has_prefix(x, 'https://')) 
datas['has_http'] = datas['url'].apply(lambda x: has_prefix(x, 'http://')) 
datas['has_ftp'] = datas['url'].apply(lambda x: has_prefix(x, 'ftp://')) 
datas['has_www'] = datas['url'].apply(lambda x: has_prefix(x, 'www.'))
print(datas)

                                                      url status  has_https  \
0       nobell.it/70ffb52d079109dca5664cce6f317373782/...     -1          0   
1       www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...     -1          0   
2       serviciosbys.com/paypal.cgi.bin.get-into.herf....     -1          0   
3       mail.printakid.com/www.online.americanexpress....     -1          0   
4       thewhiskeydregs.com/wp-content/themes/widescre...     -1          0   
...                                                   ...    ...        ...   
957367        http://ecct-it.com/docmmmnn/aptgd/index.php     -1          0   
957368  http://faboleena.com/js/infortis/jquery/plugin...     -1          0   
957369  http://faboleena.com/js/infortis/jquery/plugin...     -1          0   
957370                             http://atualizapj.com/     -1          0   
957371  http://writeassociate.com/test/Portal/inicio/I...     -1          0   

        has_http  has_ftp  has_www  
0             

In [167]:
# Función para estandarizar las urls # funcion para eliminar espacios blanco al inicio y final de las urls
def clean_url(url): 
    url= re.sub(r'\s+', '', str(url)) #eliminar espacios blancos al inicio y final de la url
    prefixes=['https://', 'http://', 'ftp://', 'www.']
    
    for prefix in prefixes: 
        while url.startswith(prefix):
            url=url[len(prefix):]
    return url
#aplicar la funcion para limpiar las url
datas['url']=datas['url'].apply(clean_url)

#funcion para verificar si hay urls q comiencen con www
def has_www(df):
    return df[df['url'].str.startswith('www.')].shape[0]

#bucle para eliminar 'www.' hasta q no queden url q empiecen cn ese prefijo
while has_www(datas)>0:
    datas['url']=datas['url'].apply(clean_url)
#verificar q todas las url han sido eliminadas
suma_filas_www= has_www(datas)
print("la suma de filas que empiezan con www. es:")
print(suma_filas_www)



la suma de filas que empiezan con www. es:
0


In [ ]:
# Eliminamos filas duplicadas en 'datas'
for df in [datas]:
    duplicados= df.duplicated().sum()
    print(f"Hay {duplicados} filas duplicadas")
    df.drop_duplicates(inplace=True)
    eliminado=duplicados
    print(f"Se ha eliminado {eliminado} filas")
    print(f"Datas sin filas duplicadas")

Hay 6536 filas duplicadas
Se ha eliminado 6536 filas
Datas sin filas duplicadas


In [169]:


#Se añade columnas
#definir el alfabeto y otros caracteres
alfabeto_minuscula= string.ascii_lowercase
alfabeto_mayuscula=string.ascii_uppercase
caracteres_str="!@#$%^&*()_+-=[]{}|;:',.<>?/\\"
caracteres=list(caracteres_str)
numeros="0123456789"

#funcion para contar las concurrencias de cada caracter
def count_chars(url,chars):
    return{chars:url.count(char)for char in chars}

#funcion para contar caracteres repetidos
def caracteres_repet(url):
    counts={}
    for char in url:
        if char in counts:
            counts[char] +=1
        else:
            counts[char]=1
        return sum(count for count in counts.values() if count>1)

#funcion para contar digitos 
def cuenta_digitos_rep(url):
    digitos = {} 
    for char in url: 
        if char.isdigit():
            if char in digitos: 
                digitos[char] += 1 
            else: 
                digitos[char] = 1 
    return digitos 

# Añadir columna para caracteres repetidos consecutivos (ejemplo: aa, bb, //)
def caracteres_repet(url): 
    count = 0 
    for i in range(1, len(url)): 
        if url[i] == url[i-1]: 
            count += 1 
    return count
datas['repeated_chars'] = datas['url'].apply(caracteres_repet) 

# Añadir columna para espacios vacíos 
#funcion para contar los espacios en una url
def count_spaces(url):
    return url.count (' ')
datas['space_count'] = datas['url'].apply(count_spaces)


#añadir columna longitud de la url
datas['url_lenght']=datas['url'].apply(len)

#añadir numero de caracteres especiales
datas['num_special_chars']=datas['url'].apply(lambda x: sum(x.count(c) for c in caracteres))
#añadir columnas para cada letra en minuscula
for char in alfabeto_minuscula:
    datas[f'count_{char}_lower'] = datas['url'].apply(lambda x: x.count(char))
#añadir columnas para cada letra en mayuscula
for char in alfabeto_mayuscula:
    datas[f'count_{char}_upper'] = datas['url'].apply(lambda x: x.count(char))

#añadir columnas para cada numeor
for num in numeros:
    datas[num]=datas['url'].apply(lambda x: x.count(num))

#añadir columnas para caracteres
for char in caracteres:
    col_name=f"count_{char}"
    datas[col_name]=datas['url'].apply(lambda x: x.count(char))



# Añadir columna de número de subdominios 
#funcion para contar los subdominios en una url
def count_subdomains(url):
    url=re.sub(r'^(http://|https://|ftp://|www\.)', '', url)
    return url.count('.')
datas['num_subdomains'] = datas['url'].apply(lambda x: x.count('.')) 



#añade columna has_ip
# Función para verificar si una URL comienza con una dirección IP 
def starts_with_ip(url): 
    # Expresión regular para encontrar direcciones IP 
    ip_pattern = re.compile(r'^(\d{1,3}\.){3}\d{1,3}') 
    return 1 if ip_pattern.match(url) else 0 
#se añade has_ip
datas['has_ip'] = datas['url'].apply(starts_with_ip)


#movemos la columna 'status' al final de las columnas 
columns = [col for col in datas.columns if col != 'status'] + ['status'] 
datas =datas[columns]
print(datas.head())    

C:\Users\Marisol\AppData\Local\Temp\ipykernel_3864\1804463928.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  datas[col_name]=datas['url'].apply(lambda x: x.count(char))
C:\Users\Marisol\AppData\Local\Temp\ipykernel_3864\1804463928.py:78: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  datas['num_subdomains'] = datas['url'].apply(lambda x: x.count('.'))
C:\Users\Marisol\AppData\Local\Temp\ipykernel_3864\1804463928.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

                                                 url  has_https  has_http  \
0  nobell.it/70ffb52d079109dca5664cce6f317373782/...          0         0   
1  dghjdgf.com/paypal.co.uk/cycgi-bin/webscrcmd=_...          0         0   
2  serviciosbys.com/paypal.cgi.bin.get-into.herf....          0         0   
3  mail.printakid.com/www.online.americanexpress....          0         0   
4  thewhiskeydregs.com/wp-content/themes/widescre...          0         0   

   has_ftp  has_www  repeated_chars  space_count  url_lenght  \
0        0        0              10            0         225   
1        0        1               0            0          77   
2        0        0               3            0         177   
3        0        0               3            0          60   
4        0        0               3            0         116   

   num_special_chars  count_a_lower  ...  count_,  count_.  count_<  count_>  \
0                 32             10  ...        0        6        0     

In [191]:
import re
#filtramos las urls con direccion ip
ip_pattern = re.compile(r'^(\d{1,3}\.){3}\d{1,3}')
filtered = datas[datas['url'].str.match(ip_pattern)]

# Contar el número total de duplicados 
duplicados = filtered[filtered.duplicated(subset='url', keep=False)]
conteo_duplicados = duplicados['url'].value_counts()
total_duplicados = conteo_duplicados.sum() 
print(f"Total de URLs duplicadas: {total_duplicados}")

#eliminado duplicados del data
datas_actualizado= datas.drop_duplicates(subset='url', keep='first')

# actualizar el data original sin las urls duplicadas
datos_actualizado= datas[~datas.index.isin(duplicados.index)]
#verificamos q se han eliminado
ip_pattern2 = re.compile(r'^(\d{1,3}\.){3}\d{1,3}')
filtered2 = datos_actualizado[datos_actualizado['url'].str.match(ip_pattern)]
conteo_duplicados2= filtered2.duplicated(subset='url', keep=False).sum()
print("verificando se tiene filas repetidas filtrado por ip")
print(conteo_duplicados2)


Total de URLs duplicadas: 7330
verificando se tiene filas repetidas filtrado por ip
0


In [193]:
datos_actualizado.to_csv('datos1.csv', index=False)

In [194]:
datos= pd.read_csv('datos1.csv')
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943506 entries, 0 to 943505
Columns: 103 entries, url to status
dtypes: int64(102), object(1)
memory usage: 741.4+ MB
